In [1]:
import pandas as pd
from tensorflow import keras
import numpy as np
from sklearn import preprocessing
from sklearn import model_selection
from pickle import load

from training_functions import summarize_data, get_encoded_labels_overshoot_undershoot_classification

In [2]:
# Load the existing model
# For testing existing models on new data
model = keras.models.load_model("../Finished Neural Network Models/OUID_3C_Optimized/OUID_3C_Optimized")
scaler = load(open('../Finished Neural Network Models/OUID_3C_Optimized/scaler.pkl','rb'))

In [3]:
# Training was performed on models for which the magnitude of the zeros and poles were no greater than 10
# Test the models on a set of data for which the magnitude of the zeros and poles can be up to 100 to see
# how the perfomance is affected
dataset = pd.read_csv("../Datasets/Three Coefficient 2nd Order/Larger Value Dataset 1.csv")
labels = get_encoded_labels_overshoot_undershoot_classification(dataset)
data_array = dataset.to_numpy()
scaled_input = scaler.transform(data_array[:,0:3])
raw_predictions = model.predict(scaled_input)
predictions = np.argmax(raw_predictions,1)
actual = np.argmax(labels,1)
# Analysis of the performance of the model
incorrect_indices = (predictions - actual).nonzero()
num_incorrect = incorrect_indices[0].shape[0]
print("Number of classification errors: " + str(num_incorrect))
print("Accuracy: " + str((dataset.shape[0] - num_incorrect)/dataset.shape[0]))
# Generate confusion matrix; rows correspond to actual label and columns correspond to predicted label
confusion_matrix = np.zeros((3,3))
for i in range(dataset.shape[0]):
    if actual[i] == 0 and predictions[i] == 0:
        confusion_matrix[0,0] = confusion_matrix[0,0] + 1
    elif actual[i] == 1 and predictions[i] == 0:
        confusion_matrix[1,0] = confusion_matrix[1,0] + 1
    elif actual[i] == 2 and predictions[i] == 0:
        confusion_matrix[2,0] = confusion_matrix[2,0] + 1
    elif actual[i] == 0 and predictions[i] == 1:
        confusion_matrix[0,1] = confusion_matrix[0,1] + 1
    elif actual[i] == 1 and predictions[i] == 1:
        confusion_matrix[1,1] = confusion_matrix[1,1] + 1
    elif actual[i] == 2 and predictions[i] == 1:
        confusion_matrix[2,1] = confusion_matrix[2,1] + 1
    elif actual[i] == 0 and predictions[i] == 2:
        confusion_matrix[0,2] = confusion_matrix[0,2] + 1
    elif actual[i] == 1 and predictions[i] == 2:
        confusion_matrix[1,2] = confusion_matrix[1,2] + 1
    elif actual[i] == 2 and predictions[i] == 2:
        confusion_matrix[2,2] = confusion_matrix[2,2] + 1
print(confusion_matrix)

3125/3125 [==============================] - 2s 673us/step
Number of classification errors: 35920
Accuracy: 0.6408
[[19825. 12248.  1261.]
 [  170. 11264. 21898.]
 [  175.   168. 32991.]]


The accuracy is nearly reduced to 50% when the magnitude of the poles/zeros is allowed to be 10 times larger, so clearly the neural network classifier cannot be applied to that data without being retrained.